In [1]:
pip install xgboost

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import pickle
from sklearn.pipeline import Pipeline

## Loading the data

In [3]:
data=pd.read_csv('data.csv')

In [4]:
data.head()

,Unnamed: 0,flair,title,score,id,url,comms_num,body,author,comments,timestamp,combined_features
0,0,Business/Finance,Interest PF taxable: Finance minister eye PF i...,51,laoy5g,https://timesofindia.indiatimes.com/business/i...,20,NaN,satyasys,"PF return fully taxable, point investing PF. G...",2021-02-02 19:30:37,Interest PF taxable: Finance minister eye PF i...
1,1,Food,How survive 500rs(food) 2 weeks?,55,kr3ztg,https://www.reddit.com/r/india/comments/kr3ztg...,79,Hey guys. So time salary going late I'll recei...,Luc_90,"1. 2 kg cheap rice, 50/kg so, (Don't buy boile...",2021-01-06 07:40:59,How survive 500rs(food) 2 weeks?1. 2 kg cheap ...
2,2,Scheduled,"Right wing group labelling resource document ""...",143,lbwl1r,https://www.reddit.com/r/india/comments/lbwl1r...,17,"Recently Greta Thunberg tweeted ""toolkit"" peop...",gobargorab,Anything father modi make go crazy upset. That...,2021-02-04 09:27:18,"Right wing group labelling resource document ""..."
3,3,Food,Ask: What amount pocket money give children?,5,m0auzn,https://www.reddit.com/r/india/comments/m0auzn...,39,"Also, supposed buy pocket money actually spend...",what_is_inflation,You guy get pocket money? My parent never gave...,2021-03-08 21:25:43,Ask: What amount pocket money give children?Yo...
4,4,Photography,"I’ve Recently generated interest photography, ...",0,aaakn4,https://i.redd.it/9tcehs8vz0721.jpg,14,NaN,thesarcasticpage,A photo like letter viewer story want convey. ...,2018-12-29 03:35:37,"I’ve Recently generated interest photography, ..."


In [5]:
data.fillna("",inplace = True)

In [6]:
data.shape

(2280, 12)

In [7]:
data.columns

Index(['Unnamed: 0', 'flair', 'title', 'score', 'id', 'url', 'comms_num',
       'body', 'author', 'comments', 'timestamp', 'combined_features'],
      dtype='object')

In [8]:
data.drop(['Unnamed: 0','id'],axis=1,inplace=True)

In [9]:
data.head()

,flair,title,score,url,comms_num,body,author,comments,timestamp,combined_features
0,Business/Finance,Interest PF taxable: Finance minister eye PF i...,51,https://timesofindia.indiatimes.com/business/i...,20,,satyasys,"PF return fully taxable, point investing PF. G...",2021-02-02 19:30:37,Interest PF taxable: Finance minister eye PF i...
1,Food,How survive 500rs(food) 2 weeks?,55,https://www.reddit.com/r/india/comments/kr3ztg...,79,Hey guys. So time salary going late I'll recei...,Luc_90,"1. 2 kg cheap rice, 50/kg so, (Don't buy boile...",2021-01-06 07:40:59,How survive 500rs(food) 2 weeks?1. 2 kg cheap ...
2,Scheduled,"Right wing group labelling resource document ""...",143,https://www.reddit.com/r/india/comments/lbwl1r...,17,"Recently Greta Thunberg tweeted ""toolkit"" peop...",gobargorab,Anything father modi make go crazy upset. That...,2021-02-04 09:27:18,"Right wing group labelling resource document ""..."
3,Food,Ask: What amount pocket money give children?,5,https://www.reddit.com/r/india/comments/m0auzn...,39,"Also, supposed buy pocket money actually spend...",what_is_inflation,You guy get pocket money? My parent never gave...,2021-03-08 21:25:43,Ask: What amount pocket money give children?Yo...
4,Photography,"I’ve Recently generated interest photography, ...",0,https://i.redd.it/9tcehs8vz0721.jpg,14,,thesarcasticpage,A photo like letter viewer story want convey. ...,2018-12-29 03:35:37,"I’ve Recently generated interest photography, ..."


In [10]:
data['flair'].value_counts()

Politics              244
Coronavirus           241
Food                  240
Scheduled             234
Business/Finance      230
AskIndia              226
Science/Technology    220
Photography           219
Policy/Economy        214
Non-Political         212
Name: flair, dtype: int64

In [11]:
flair = ['Politics','Coronavirus','Food','Scheduled','Business/Finance','AskIndia','Science/Technology'
        'Photography','Policy/Economy','Non-Political']

## Trying different models

## SVM

In [12]:
def linear_svm(X_train, X_test, y_train, y_test):
    sgd = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', SGDClassifier(loss='hinge',alpha=1e-4, random_state=16, max_iter=5, tol=None)),
                 ])
    sgd.fit(X_train, y_train)

    y_pred = sgd.predict(X_test)

    print('accuracy %s' % accuracy_score(y_pred, y_test))
    print(classification_report(y_test, y_pred,labels=flair))

## Random Forest

In [13]:
def randomforest(X_train, X_test, y_train, y_test):
    ranfor = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', RandomForestClassifier(n_estimators = 1000, random_state = 42)),
                 ])
    ranfor.fit(X_train, y_train)

    y_pred = ranfor.predict(X_test)

    print('accuracy %s' % accuracy_score(y_pred, y_test))
    print(classification_report(y_test, y_pred,labels=flair))

## XG Boost

In [14]:
def xgbclassifier(X_train, X_test, y_train, y_test):  
    xgb_clf = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', XGBClassifier(random_state=42, seed=2,n_estimators=1000,verbosity=1,objective='multi:softmax')),
                 ])
    xgb_clf.fit(X_train, y_train)

    y_pred = xgb_clf.predict(X_test)

    print('accuracy %s' % accuracy_score(y_pred, y_test))
    print(classification_report(y_test, y_pred,labels=flair))

## Logistic Regression

In [15]:
def logisticreg(X_train, X_test, y_train, y_test):

    from sklearn.linear_model import LogisticRegression

    logreg = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                 ])
    logreg.fit(X_train, y_train)

    y_pred = logreg.predict(X_test)

    print('accuracy %s' % accuracy_score(y_pred, y_test))
    print(classification_report(y_test, y_pred,labels=flair))

## Evaluating the model

In [16]:
def train_test(X,y):
 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
    print("Results of Linear Support Vector Machine")
    linear_svm(X_train, X_test, y_train, y_test)
    print("Results of Logistic Regression\n")
    logisticreg(X_train, X_test, y_train, y_test)
    print("Results of Random Forest\n")
    randomforest(X_train, X_test, y_train, y_test)
    print("Results of XGB Classifier\n")
    xgbclassifier(X_train, X_test, y_train, y_test)

In [17]:
cat = data['flair']

V = data['combined_features']
W = data['comments']
X = data['title']
Y = data['body']
Z = data['url']

print("Flair Detection using Title as Feature\n")
train_test(X,cat)
print("Flair Detection using Body as Feature")
train_test(Y,cat)
print("Flair Detection using URL as Feature")
train_test(Z,cat)
print("Flair Detection using Comments as Feature")
train_test(W,cat)
print("Flair Detection using Combined Features")
train_test(V,cat)

Flair Detection using Title as Feature

Results of Linear Support Vector Machine
accuracy 0.6491228070175439
                               precision    recall  f1-score   support

                     Politics       0.57      0.66      0.61        47
                  Coronavirus       0.85      0.85      0.85        59
                         Food       0.67      0.57      0.61        46
                    Scheduled       0.58      0.59      0.59        49
             Business/Finance       0.40      0.45      0.43        42
                     AskIndia       0.81      0.74      0.78        47
Science/TechnologyPhotography       0.00      0.00      0.00         0
               Policy/Economy       0.43      0.40      0.42        47
                Non-Political       0.92      0.97      0.95        36

                    micro avg       0.65      0.65      0.65       373
                    macro avg       0.58      0.58      0.58       373
                 weighted avg       0

C:\Users\saksh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\saksh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\saksh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative

accuracy 0.6557017543859649
                               precision    recall  f1-score   support

                     Politics       0.68      0.64      0.66        47
                  Coronavirus       0.78      0.86      0.82        59
                         Food       0.71      0.70      0.70        46
                    Scheduled       0.64      0.55      0.59        49
             Business/Finance       0.40      0.43      0.41        42
                     AskIndia       0.81      0.72      0.76        47
Science/TechnologyPhotography       0.00      0.00      0.00         0
               Policy/Economy       0.43      0.47      0.45        47
                Non-Political       0.89      0.94      0.92        36

                    micro avg       0.67      0.66      0.67       373
                    macro avg       0.60      0.59      0.59       373
                 weighted avg       0.67      0.66      0.67       373

Results of Random Forest

accuracy 0.673245614

C:\Users\saksh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\saksh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\saksh\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [n

[21:58:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
accuracy 0.6491228070175439
                               precision    recall  f1-score   support

                     Politics       0.53      0.64      0.58        47
                  Coronavirus       0.82      0.86      0.84        59
                         Food       0.76      0.63      0.69        46
                    Scheduled       0.64      0.51      0.57        49
             Business/Finance       0.41      0.45      0.43        42
                     AskIndia       0.83      0.74      0.79        47
Science/TechnologyPhotography       0.00      0.00      0.00         0
               Policy/Economy       0.42      0.45      0.43        47
                Non-P

...

C:\Users\saksh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\saksh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy 0.8048245614035088
                               precision    recall  f1-score   support

                     Politics       0.65      0.74      0.69        47
                  Coronavirus       0.88      0.95      0.91        59
                         Food       0.90      0.78      0.84        46
                    Scheduled       0.88      0.76      0.81        49
             Business/Finance       0.64      0.86      0.73        42
                     AskIndia       0.81      0.83      0.82        47
Science/TechnologyPhotography       0.00      0.00      0.00         0
               Policy/Economy       0.79      0.57      0.67        47
                Non-Political       0.84      0.89      0.86        36

                    micro avg       0.79      0.80      0.80       373
                    macro avg       0.71      0.71      0.70       373
                 weighted avg       0.80      0.80      0.80       373

Results of XGB Classifier



C:\Users\saksh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\saksh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\saksh\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [n

[22:16:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
accuracy 0.8486842105263158
                               precision    recall  f1-score   support

                     Politics       0.69      0.70      0.69        47
                  Coronavirus       0.98      0.95      0.97        59
                         Food       0.80      0.78      0.79        46
                    Scheduled       0.91      0.84      0.87        49
             Business/Finance       0.78      0.86      0.82        42
                     AskIndia       0.93      0.89      0.91        47
Science/TechnologyPhotography       0.00      0.00      0.00         0
               Policy/Economy       0.77      0.79      0.78        47
                Non-P

C:\Users\saksh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\saksh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Saving the model

In [18]:
import xgboost as xgb
X_train, X_test, y_train, y_test = train_test_split(V, cat, test_size=0.2, random_state = 42)
model = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                   ('clf', xgb.sklearn.XGBClassifier(random_state=42,n_estimators=1000,verbosity=1, seed=2,objective='multi:softmax')),
                  ])
XGB = model.fit(X_train, y_train)

C:\Users\saksh\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:26:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [21]:
# Saving the best model as pickle file which would be used in website

with open("pickle_xgboost_model.pkl", 'wb') as file:  
    pickle.dump(XGB, file)